## Install huggingface transformers

In [1]:
!pip install transformers
!pip install pytorch-nlp


    100% |████████████████████████████████| 573kB 24.6MB/s ta 0:00:01
    100% |████████████████████████████████| 890kB 25.1MB/s ta 0:00:01
    100% |████████████████████████████████| 686kB 29.3MB/s ta 0:00:01
    100% |████████████████████████████████| 1.0MB 26.3MB/s ta 0:00:01
    100% |████████████████████████████████| 3.7MB 11.3MB/s ta 0:00:01
    100% |████████████████████████████████| 296kB 40.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for sacremoses ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 92kB 16.6MB/s ta 0:00:01
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however 

In [2]:
import torch                                        # root package
from torch.utils.data import Dataset, DataLoader    # dataset representation and loading


In [3]:
import os
import boto3
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
print(role)

arn:aws:iam::733425554560:role/service-role/AmazonSageMaker-ExecutionRole-20200504T094270


## 1. Obtain dataset

### We don't want to use the prepared dataset as it, wo we compare the sample dataset into csv

In [4]:
import csv
from torchnlp.datasets import imdb_dataset
train, test = imdb_dataset(train=True,test=True)

with open('data/train.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['text','sentiment'])
    for i in train:
        csvwriter.writerow([i['text'],i['sentiment']])

with open('data/test.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['text','sentiment'])
    for i in test:
        csvwriter.writerow([i['text'],i['sentiment']])

In [5]:
import pandas as pd
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
mapping = {'neg': -1, 'pos': 1}
train_df = train_df.replace({'sentiment': mapping})

In [6]:
train_df.sample(10)

,text,sentiment
17216,Starring an unknown cast which seem likely to ...,-1
378,I lived during those times and I think the pro...,1
5183,"In ten words or less to describe this film, Ba...",1
20622,Despite being released on DVD by Blue Undergro...,-1
20068,I am compelled to write a review of this IMAX ...,-1
6354,Opening credits: great. Music: just right for ...,1
20657,"There are bad movies, terrible movies even bor...",-1
14948,Given the history of the director of this movi...,-1
17094,"When I first tuned in on this morning news, I ...",-1
21138,This movie causes more unintentional laughter ...,-1


In [7]:
texts = train_df.text.values
sentiments = train_df.sentiment.values

In [8]:
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer
torch.set_grad_enabled(False)

In [9]:
# Store the model we want to use
MODEL_NAME = "bert-base-uncased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

In [10]:
tokens = tokenizer.tokenize(train[0]['text'])
print("Tokens: {}".format(tokens))

Tokens: ['i', 'really', 'liked', 'this', 'summers', '##lam', 'due', 'to', 'the', 'look', 'of', 'the', 'arena', ',', 'the', 'curtains', 'and', 'just', 'the', 'look', 'overall', 'was', 'interesting', 'to', 'me', 'for', 'some', 'reason', '.', 'anyway', '##s', ',', 'this', 'could', 'have', 'been', 'one', 'of', 'the', 'best', 'summers', '##lam', "'", 's', 'ever', 'if', 'the', 'wwf', 'didn', "'", 't', 'have', 'lex', 'lu', '##ger', 'in', 'the', 'main', 'event', 'against', 'yoko', '##zu', '##na', ',', 'now', 'for', 'it', "'", 's', 'time', 'it', 'was', 'ok', 'to', 'have', 'a', 'huge', 'fat', 'man', 'vs', 'a', 'strong', 'man', 'but', 'i', "'", 'm', 'glad', 'times', 'have', 'changed', '.', 'it', 'was', 'a', 'terrible', 'main', 'event', 'just', 'like', 'every', 'match', 'lu', '##ger', 'is', 'in', 'is', 'terrible', '.', 'other', 'matches', 'on', 'the', 'card', 'were', 'razor', 'ramon', 'vs', 'ted', 'di', '##bia', '##se', ',', 'steiner', 'brothers', 'vs', 'heavenly', 'bodies', ',', 'shawn', 'michael

In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [12]:
print('Original: ',texts[0])
print('Tokenized: ',tokenizer.tokenize(texts[0]))
print('Token IDs: ',tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))

Original:  I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok to have a huge fat man vs a strong man but I'm glad times have changed. It was a terrible main event just like every match Luger is in is terrible. Other matches on the card were Razor Ramon vs Ted Dibiase, Steiner Brothers vs Heavenly Bodies, Shawn Michaels vs Curt Hening, this was the event where Shawn named his big monster of a body guard Diesel, IRS vs 1-2-3 Kid, Bret Hart first takes on Doink then takes on Jerry Lawler and stuff with the Harts and Lawler was always very interesting, then Ludvig Borga destroyed Marty Jannetty, Undertaker took on Giant Gonzalez in another terrible match, The Smoking Gunns and Tatanka took on Bam Bam Bigelow and the Headshrinkers, and Yokozuna d

### Tokenize Dataset

In [13]:
input_ids = []
attention_masks = []

for t in texts:
    encoded_dict = tokenizer.encode_plus(
                        t,
                        add_special_tokens = True,
                        max_length = 50,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    
input_ids = torch.cat(input_ids, dim=0)
attention_mask = torch.cat(attention_masks, dim=0)
labels = torch.tensor(sentiments)

print('Original: ', texts[0])
print('Token IDs: ', input_ids[0])

Original:  I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok to have a huge fat man vs a strong man but I'm glad times have changed. It was a terrible main event just like every match Luger is in is terrible. Other matches on the card were Razor Ramon vs Ted Dibiase, Steiner Brothers vs Heavenly Bodies, Shawn Michaels vs Curt Hening, this was the event where Shawn named his big monster of a body guard Diesel, IRS vs 1-2-3 Kid, Bret Hart first takes on Doink then takes on Jerry Lawler and stuff with the Harts and Lawler was always very interesting, then Ludvig Borga destroyed Marty Jannetty, Undertaker took on Giant Gonzalez in another terrible match, The Smoking Gunns and Tatanka took on Bam Bam Bigelow and the Headshrinkers, and Yokozuna d

In [17]:
from transformers import BertModel

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# If you have a GPU, put everything on cuda
input_ids = input_ids.to('cuda')
labels = labels.to('cuda')
model.to('cuda')


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          